In [10]:
import numpy as np
import pandas as pd
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.porter import PorterStemmer

In [1]:
movies_file_path = r'D:\movie-recommend\tmdb_5000_movies.csv\tmdb_5000_movies.csv'
credits_file_path = r'D:\movie-recommend\tmdb_5000_credits.csv\tmdb_5000_credits.csv'

In [4]:
movies = pd.read_csv(movies_file_path)
credits = pd.read_csv(credits_file_path)

In [12]:
movie = movies.merge(credits, on='title')
movie = movie[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]
movie.dropna(inplace=True)

In [13]:
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return 

In [7]:
def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter += 1
        else:
            break
    return L


In [8]:
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L


In [14]:
movie['genres'] = movie['genres'].apply(convert)
movie['keywords'] = movie['keywords'].apply(convert)
movie['cast'] = movie['cast'].apply(convert3)
movie['crew'] = movie['crew'].apply(fetch_director)
movie['overview'] = movie['overview'].apply(lambda x: x.split())
movie['keywords'] = movie['keywords'].apply(lambda x: [i.replace(" ", "") for i in x])
movie['genres'] = movie['genres'].apply(lambda x: [i.replace(" ", "") for i in x])
movie['cast'] = movie['cast'].apply(lambda x: [i.replace(" ", "") for i in x])
movie['crew'] = movie['crew'].apply(lambda x: [i.replace(" ", "") for i in x])
movie['tags'] = movie['overview'] + movie['genres'] + movie['keywords'] + movie['cast'] + movie['crew']

TypeError: 'NoneType' object is not iterable

In [15]:
new_df = movie[['movie_id', 'title', 'tags']]
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))

KeyError: "['tags'] not in index"

In [ ]:
ps = PorterStemmer()

In [ ]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)
new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())

In [ ]:
cv = CountVectorizer(max_features=5000, stop_words='english')
vectors = cv.fit_transform(new_df['tags']).toarray()
similarity = cosine_similarity(vectors)

In [ ]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]

    recommended_movies = [new_df.iloc[i[0]].title for i in movies_list]
    return recommended_movies